# 구글드라이브

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# whisper

In [2]:
# Whisper 설치
!pip install -q git+https://github.com/openai/whisper.git

# 오디오 처리를 위한 FFmpeg 설치 (Colab 환경에 따라 필요할 수 있음)
# Colab 기본 환경에는 보통 설치되어 있지만, 확실하게 하기 위해 실행
!apt update && apt install -y ffmpeg

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,102 kB]
Get:1

In [3]:
# 필요한 라이브러리 설치
!pip install -q transformers accelerate datasets soundfile pandas

# 오디오 처리를 위한 FFmpeg 설치 (Colab 환경에 따라 필요할 수 있음)
!apt update && apt install -y ffmpeg

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
42 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

In [7]:
from google.colab import drive
import os
import pandas as pd # pandas를 여기서 미리 임포트

# Google Drive 마운트
drive.mount('/content/drive')

# 변환하려는 오디오 파일 경로 설정
audio_file_path = "/content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a"
# CSV 파일을 저장할 폴더 경로 (원본 파일과 같은 폴더)
output_dir = os.path.dirname(audio_file_path)

print(f"설정된 파일 경로: {audio_file_path}")
print(f"CSV 저장 폴더: {output_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
설정된 파일 경로: /content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a
CSV 저장 폴더: /content/drive/MyDrive/final_project/회의록


In [8]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


# ---------------------------------------------
# 🛠️ Pipeline 설정
# ---------------------------------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# 사용할 모델 ID 지정
model_id = "openai/whisper-large-v3"

# 모델 이름에서 슬래시(/)를 제거하여 파일 이름에 사용할 깔끔한 모델 이름 생성
model_name_for_file = model_id.split("/")[-1]

print(f"사용 장치: {device}, dtype: {torch_dtype}, 모델: {model_id}")

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# 💡 'return_timestamps' 옵션을 제거하거나 "false"로 설정하여 세그먼트 추출 부담을 줄입니다.
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# ---------------------------------------------
# 🎤 음성 인식 실행
# ---------------------------------------------
print(f"'{audio_file_path}' 파일 변환 시작...")
result = pipe(
    audio_file_path,
    chunk_length_s=30,
    stride_length_s=(4, 2),
    generate_kwargs={"language": "ko"},
)

# ---------------------------------------------
# 💾 인식 결과를 텍스트만 CSV 파일로 저장
# ---------------------------------------------

# 1. 저장할 전체 텍스트 추출
full_text = result['text'].strip()

# 2. CSV 파일 이름 및 경로 설정
# 파일 이름: 모델 이름 (예: whisper-large-v3) + .csv
csv_file_name = f"{model_name_for_file}.csv"
csv_file_path = os.path.join(output_dir, csv_file_name)

# 3. Pandas DataFrame 생성 (컬럼 하나만 사용)
df_text = pd.DataFrame({
    'transcribed_text': [full_text]
})

# 4. CSV 파일로 저장
df_text.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# 결과 출력
print("--- 📝 전체 인식 결과 ---")
print(full_text)
print("------------------------")
print(f"\n✅ CSV 저장 완료: '{csv_file_path}'")
print(f"저장된 데이터프레임 미리보기:\n{df_text.head()}")

사용 장치: cuda:0, dtype: torch.float16, 모델: openai/whisper-large-v3


Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


'/content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a' 파일 변환 시작...
--- 📝 전체 인식 결과 ---
시작되면 말을 하시죠. 시작. 네, 눌렀습니다. 다시 제 것부터 간략히 말씀드리면 저는 3개의 주제에 대해서 찾아보고 있었는데요. 어제는 주로 저는 작가 문체로 자동 교정해주는 것을 염두에 두고 시작해봤는데 결과적으로 다 마음에 들지만 데이터도 쉽게 구한다는 말은 쉽게 꿀빠하려는 의미보다는 쉽게 더 강력한 데이터를 구해서 우리 모델한테 더 좋은 먹이를 주고 싶다는 연료를 주고 싶다는 마음이 컸어요. 그러다 보니까 실시간 세계 경제 뉴스나 바로 하자 자동 태깅 쪽으로 갈 수밖에 없는 판단을 했고요. 자발적으로. 그러면 그 두 개 중에 제가 힘을 싣고 싶었던 거는 좀 더 어필링한 포트폴리오를 위해서 바로 하자 자동탱킹을 좀 더 해보자. 그리고 팀원들의 어떤 의견 수렴도 전반적 그런 기류였던 것 같아서 같이 그냥 동참해서 가는 게 좋을 것 같아서 신속한 의사결정을 위해서 자료를 조사해봤더니 국립국어원에서 상당히 강력하고 풍부한 데이터를 제공해 주는 것 같습니다. 이게 지금 음성 형태인지 아니면 그냥 포퍼스라니까 그냥 어떤 텍스트 형태인지 좀 더 뜯어보고 공유 드릴 예정이고요. 그리고 두 번째로 제가 염두에 두는 게 AI 허브의 대화 데이터는 정말 무궁무진하고 많더라고요. 그리고 상황도 다양해요. 예를 들면 긴급 출동 상황, 공항 주변 소음 상황, 공사장 상황, 그리고 그런 어떤 노이즈가 포함됐냐 안 됐냐를 떠나서도 엄청 많더라고요. 아동의 대화 이런 거. 그래서 우리에게 필요한 데이터로 보여지는 게 많아서 좀 많이 스크랩 해놨으니까 이따 또 한번 리스트업해서 공유하도록 하고 어쨌든 그래서 결과적으로 저는 바라자 자동 태깅 쪽으로 데이터를 좀 많이 구비해놨다 정도로 볼 수 있고 여기서 이런 데이터는 충분하네요 라고 해서 되면 바로 파이프라인 설계로 넘어가도 저는 괜찮을 것 같습니다. 다른 분들? 저부터 해야 되나

In [9]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# ---------------------------------------------
# 🛠️ Pipeline 설정
# ---------------------------------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# 사용할 모델 ID 지정
model_id = "openai/whisper-large-v3-turbo"

# 모델 이름에서 슬래시(/)를 제거하여 파일 이름에 사용할 깔끔한 모델 이름 생성
model_name_for_file = model_id.split("/")[-1]

print(f"사용 장치: {device}, dtype: {torch_dtype}, 모델: {model_id}")

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# 💡 'return_timestamps' 옵션을 제거하거나 "false"로 설정하여 세그먼트 추출 부담을 줄입니다.
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# ---------------------------------------------
# 🎤 음성 인식 실행
# ---------------------------------------------
print(f"'{audio_file_path}' 파일 변환 시작...")
result = pipe(
    audio_file_path,
    chunk_length_s=30,
    stride_length_s=(4, 2),
    generate_kwargs={"language": "ko"},
)

# ---------------------------------------------
# 💾 인식 결과를 텍스트만 CSV 파일로 저장
# ---------------------------------------------

# 1. 저장할 전체 텍스트 추출
full_text = result['text'].strip()

# 2. CSV 파일 이름 및 경로 설정
# 파일 이름: 모델 이름 (예: whisper-large-v3) + .csv
csv_file_name = f"{model_name_for_file}.csv"
csv_file_path = os.path.join(output_dir, csv_file_name)

# 3. Pandas DataFrame 생성 (컬럼 하나만 사용)
df_text = pd.DataFrame({
    'transcribed_text': [full_text]
})

# 4. CSV 파일로 저장
df_text.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# 결과 출력
print("--- 📝 전체 인식 결과 ---")
print(full_text)
print("------------------------")
print(f"\n✅ CSV 저장 완료: '{csv_file_path}'")
print(f"저장된 데이터프레임 미리보기:\n{df_text.head()}")

사용 장치: cuda:0, dtype: torch.float16, 모델: openai/whisper-large-v3-turbo


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


'/content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a' 파일 변환 시작...
--- 📝 전체 인식 결과 ---
시작되면 말하시죠 쉽게 구한다는 말은 쉽게 꿀 빨으려는 의미보다는 쉽게 더 강력한 데이터를 구해서 우리 모델한테 더 좋은 먹이를 주고 싶다는 연료를 주고 싶다는 마음이 컸어요. 그래가지고 그러다 보니까 또 실시간 세계 경제 뉴스나 바라다 자동 태깅 쪽으로 갈 수밖에 없는 판단을 했고요. 자발적으로. 그래서 이제 또 그러면 이제 그 두 개 중에 좀 더 제가 힘을 싣고 싶었던 거는 좀 더 이렇게 어�링한 포트폴리오를 위해서 바로 하자 자동 탱임을 좀 더 해보자. 그리고 팀원들의 의견 수렴도 전반적으로 그런 기류였던 것 같아서 같이 그냥 동참해서 가는 게 좋을 것 같아서 신속한 의사결정을 위해서. 근데 이제 자료를 조사해 봤더니 국립국어원에서 상당히 강력하고 풍부한 데이터를 제공해 주는 것 같습니다. 근데 이게 지금 음성 형태인지 아니면 그냥 포퍼스라니까 그냥 어떤 텍스트 형태인지 좀 더 뜯어보고 공유 드릴 예정이고요. 그리고 두 번째로 제가 지금 염두에 두는 게 AI 허브의 대화 데이터는 정말 무궁무진하고 많더라고요. 그리고 상황도 다양해요. 예를 들면 긴급출동 상황, 공항 주변 소음 상황, 공사장 상황, 그런 노이즈가 포함됐냐 안 됐냐를 떠나서도 엄청 많더라고요. 아동의 대화 이런 거. 그래서 우리에게 필요한 데이터로 보여지는 게 많아서 좀 많이 스크랩 해놨으니까 이따 또 한번 리스트업해서 공유하도록 하고 어쨌든 그래서 결과적으로 저는 바라자 자동 태깅 쪽으로 데이터를 좀 많이 구비해놨다 정도로 볼 수 있고 여기서 이런 데이터는 충분하네요 라고 해서 되면 바로 파이프라인 설계로 넘어가도 저는 괜찮을 것 같습니다. 비전적 양식인 것 같아서 그 부분은 조금 고민을 안했습니다 안 할 것 같아서 저도 그래서 두번째로 저도 작가 문체를 조금 찾아봤습니다 작가 문체에 대한 데이터는 AI 허브에도 있었고 허깅페이스나 G

In [11]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# ---------------------------------------------
# 🛠️ Pipeline 설정
# ---------------------------------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# 사용할 모델 ID 지정
model_id = "openai/whisper-small"

# 모델 이름에서 슬래시(/)를 제거하여 파일 이름에 사용할 깔끔한 모델 이름 생성
model_name_for_file = model_id.split("/")[-1]

print(f"사용 장치: {device}, dtype: {torch_dtype}, 모델: {model_id}")

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# 💡 'return_timestamps' 옵션을 제거하거나 "false"로 설정하여 세그먼트 추출 부담을 줄입니다.
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# ---------------------------------------------
# 🎤 음성 인식 실행
# ---------------------------------------------
print(f"'{audio_file_path}' 파일 변환 시작...")
result = pipe(
    audio_file_path,
    chunk_length_s=30,
    stride_length_s=(4, 2),
    generate_kwargs={"language": "ko"},
)

# ---------------------------------------------
# 💾 인식 결과를 텍스트만 CSV 파일로 저장
# ---------------------------------------------

# 1. 저장할 전체 텍스트 추출
full_text = result['text'].strip()

# 2. CSV 파일 이름 및 경로 설정
# 파일 이름: 모델 이름 (예: whisper-large-v3) + .csv
csv_file_name = f"{model_name_for_file}.csv"
csv_file_path = os.path.join(output_dir, csv_file_name)

# 3. Pandas DataFrame 생성 (컬럼 하나만 사용)
df_text = pd.DataFrame({
    'transcribed_text': [full_text]
})

# 4. CSV 파일로 저장
df_text.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# 결과 출력
print("--- 📝 전체 인식 결과 ---")
print(full_text)
print("------------------------")
print(f"\n✅ CSV 저장 완료: '{csv_file_path}'")
print(f"저장된 데이터프레임 미리보기:\n{df_text.head()}")

사용 장치: cuda:0, dtype: torch.float16, 모델: openai/whisper-small


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


'/content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a' 파일 변환 시작...
--- 📝 전체 인식 결과 ---
시작되면 말하겠습니다. 네. 시작을 불렀습니다. 네. 자, 다시 이제 제 것부터 간략히 말씀드리면 저는 이제 세계 주제에 대해서 찾아보고 있었는데요. 어제는 주로 저는 작가 문제로 자동 교정해주는 것을 염두에 두고 시작해봤는데 결과적으로 다 마음에 들지만 데이터도 쉽게 구한다는 말은 쉽게 꿀발려는 의미보다는 쉽게 더 강력한 데이터를 구해서 우리 모델한테 더 좋은 먹이를 주고 싶다는 연료를 주고 싶다는 마음이 컸어요. 그러다 보니까 또 실시간 3개 경쟁 뉴스 나 바로 하자 자동 택인 쪽으로 갈 수밖에 좀 없는 판단을 했고요. 자발적으로 그래서 이제 또 그러면 이제 그 두 개 중에 좀 더 제가 힘을 실고 싶었던 건 좀 더 이렇게 어필링한 어떤 포트폴리오를 위해서 바라자 자동탱기 좀 더 해보자. 그리고 키몬들의 어떤 의견수렴도 전반적으로 그런 기류였던 것 같아서 같이 그냥 동참해서 가는 게 좋을 것 같아서 신속한 의사 결정을 해서. 그래서 이제 자를 조사해 봤더니 국립구가원에서 상당히 강력하고 풍부한 데이터를 제공해 주는 것 같습니다. 근데 이게 지금 음성 형태인지 아니면 그냥 코퍼스라니까 그냥 어떤 그 텍스트 형태인지 좀 더 뜯어 보고 공유 드릴 예정이고요. 그리고 이제 두 번째 제가 지금 염두에 두는 게 AI hub의 대화 데이터는 정말 무궁무진하고 많더라고요. 그리고 상황도 다양해요. 예를 들면 긴급출동 상황, 공항하도록 하고 어쨌든 그래서 결과적으로 저는 바라자 자동 태경 쪽으로 데이터를 많이 구비해놨다 정도 볼 수 있고 여기서 이런 데이터를 충분하네요 라고 해서 바로 파이프라인 설계로 넘어가도 저는 괜찮을 것 같습니다. 비전적인 양식인 것 같아서 그 부분은 조금 고민을 안 했습니다 안 할 것 같아서 저도 다른 그래서 두 번째로 저도 작가 문체를 조금 찾아봤습니다 작가 문체에 대한 데이터는 어느정도 AI

In [12]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# ---------------------------------------------
# 🛠️ Pipeline 설정
# ---------------------------------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# 사용할 모델 ID 지정
model_id = "openai/whisper-base"

# 모델 이름에서 슬래시(/)를 제거하여 파일 이름에 사용할 깔끔한 모델 이름 생성
model_name_for_file = model_id.split("/")[-1]

print(f"사용 장치: {device}, dtype: {torch_dtype}, 모델: {model_id}")

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# 💡 'return_timestamps' 옵션을 제거하거나 "false"로 설정하여 세그먼트 추출 부담을 줄입니다.
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# ---------------------------------------------
# 🎤 음성 인식 실행
# ---------------------------------------------
print(f"'{audio_file_path}' 파일 변환 시작...")
result = pipe(
    audio_file_path,
    chunk_length_s=30,
    stride_length_s=(4, 2),
    generate_kwargs={"language": "ko"},
)

# ---------------------------------------------
# 💾 인식 결과를 텍스트만 CSV 파일로 저장
# ---------------------------------------------

# 1. 저장할 전체 텍스트 추출
full_text = result['text'].strip()

# 2. CSV 파일 이름 및 경로 설정
# 파일 이름: 모델 이름 (예: whisper-large-v3) + .csv
csv_file_name = f"{model_name_for_file}.csv"
csv_file_path = os.path.join(output_dir, csv_file_name)

# 3. Pandas DataFrame 생성 (컬럼 하나만 사용)
df_text = pd.DataFrame({
    'transcribed_text': [full_text]
})

# 4. CSV 파일로 저장
df_text.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# 결과 출력
print("--- 📝 전체 인식 결과 ---")
print(full_text)
print("------------------------")
print(f"\n✅ CSV 저장 완료: '{csv_file_path}'")
print(f"저장된 데이터프레임 미리보기:\n{df_text.head()}")

사용 장치: cuda:0, dtype: torch.float16, 모델: openai/whisper-base


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


'/content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a' 파일 변환 시작...
--- 📝 전체 인식 결과 ---
시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 시작! 10개 구한다는 말은 쉽게 꿀빨려는 의미보다

In [13]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# ---------------------------------------------
# 🛠️ Pipeline 설정
# ---------------------------------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# 사용할 모델 ID 지정
model_id = "openai/whisper-medium"

# 모델 이름에서 슬래시(/)를 제거하여 파일 이름에 사용할 깔끔한 모델 이름 생성
model_name_for_file = model_id.split("/")[-1]

print(f"사용 장치: {device}, dtype: {torch_dtype}, 모델: {model_id}")

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# 💡 'return_timestamps' 옵션을 제거하거나 "false"로 설정하여 세그먼트 추출 부담을 줄입니다.
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# ---------------------------------------------
# 🎤 음성 인식 실행
# ---------------------------------------------
print(f"'{audio_file_path}' 파일 변환 시작...")
result = pipe(
    audio_file_path,
    chunk_length_s=30,
    stride_length_s=(4, 2),
    generate_kwargs={"language": "ko"},
)

# ---------------------------------------------
# 💾 인식 결과를 텍스트만 CSV 파일로 저장
# ---------------------------------------------

# 1. 저장할 전체 텍스트 추출
full_text = result['text'].strip()

# 2. CSV 파일 이름 및 경로 설정
# 파일 이름: 모델 이름 (예: whisper-large-v3) + .csv
csv_file_name = f"{model_name_for_file}.csv"
csv_file_path = os.path.join(output_dir, csv_file_name)

# 3. Pandas DataFrame 생성 (컬럼 하나만 사용)
df_text = pd.DataFrame({
    'transcribed_text': [full_text]
})

# 4. CSV 파일로 저장
df_text.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# 결과 출력
print("--- 📝 전체 인식 결과 ---")
print(full_text)
print("------------------------")
print(f"\n✅ CSV 저장 완료: '{csv_file_path}'")
print(f"저장된 데이터프레임 미리보기:\n{df_text.head()}")

사용 장치: cuda:0, dtype: torch.float16, 모델: openai/whisper-medium


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


'/content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a' 파일 변환 시작...
--- 📝 전체 인식 결과 ---
시작되면 말하겠습니다. 시작을 눌렀습니다. 쉽게 구한다는 말은 쉽게 꿀빠하려는 의미 보다는 쉽게 더 강력한 데이터를 구해서 우리 모델한테 더 좋은 먹이를 주고 싶다는 연료를 주고 싶다는 마음이 컸어요 그러다 보니까 또 실시간 세계 경쟁 뉴슥� 위해서 바로 하자 자동 탱이 좀 더 해보자 그리고 팀원들의 어떤 의견 수렴도 전반적으로 그런 기류였던 것 같아서 같이 그냥 동참해서 가는 게 좋을 것 같아서 신속한 의사결정을 해서 이제 자료를 제가 봤더니 국립국어원에서 상당히 강력하고 풍부한 데이터를 제공해 주는 것 같습니다 근데 이게 지금 음성 형태인지 아니면 그냥 코퍼스라니까 그냥 어떤 텍스트 형태인지 좀 더 뜯어보고 공유 드릴 예정이구요 그리고 이제 두번째로 제가 지금 염두에 두고 있는게 AI Hub의 대화 데이터는 정말 무궁무진하고 많더라구요 그리고 상황도 다양해요 예를 들면 긴급출동 상황 공항 주변 소음 상황 공사장 상황 그리고 그런 어떤 노이즈가 포함됐냐 안됐냐를 떠나서도 엄청 많더라 뭐 아동의 대화 뭐 이런거 그래서 우리가 우리에게 필요한 데이터로 보여진게 많아서 좀 이제 많이 스크랩 해놨으니까 이따 또 한번 리스트업해서 공유하도록 하고 어쨌든 그래서 결과적으로 저는 바라자 자동 태깅 쪽으로 데이터를 좀 많이 구비해놨다 정도 볼 수 있고 여기서 어 이런 데이터는 충분하네요 라고 해서 되면 바로 파이프라인 설계로 넘어가도 저는 괜찮을 것 같습니다. 그래서 두 번째로 저도 작가 문체를 조금 찾아봤습니다 작가 문체에 대한 데이터는 어느정도 AI 허브에도 있었고 허깅페이스나 기톡에도 좀 있었던 것 같아요 그래서 좀 데이터를 가져왔고 근데 이제 어떻게 좀 진행을 해야 되는가 너무 어려운 것 같아요 저는 문체를 변경한다는 학습 자체가 너무 어렵고 그걸 어떻게 진행해야 될까는 아무리 생각해서 저는 이때 실패했거든요 

In [14]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

# ---------------------------------------------
# 🛠️ Pipeline 설정
# ---------------------------------------------
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# 사용할 모델 ID 지정
model_id = "openai/whisper-large-v2"

# 모델 이름에서 슬래시(/)를 제거하여 파일 이름에 사용할 깔끔한 모델 이름 생성
model_name_for_file = model_id.split("/")[-1]

print(f"사용 장치: {device}, dtype: {torch_dtype}, 모델: {model_id}")

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# 💡 'return_timestamps' 옵션을 제거하거나 "false"로 설정하여 세그먼트 추출 부담을 줄입니다.
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

# ---------------------------------------------
# 🎤 음성 인식 실행
# ---------------------------------------------
print(f"'{audio_file_path}' 파일 변환 시작...")
result = pipe(
    audio_file_path,
    chunk_length_s=30,
    stride_length_s=(4, 2),
    generate_kwargs={"language": "ko"},
)

# ---------------------------------------------
# 💾 인식 결과를 텍스트만 CSV 파일로 저장
# ---------------------------------------------

# 1. 저장할 전체 텍스트 추출
full_text = result['text'].strip()

# 2. CSV 파일 이름 및 경로 설정
# 파일 이름: 모델 이름 (예: whisper-large-v3) + .csv
csv_file_name = f"{model_name_for_file}.csv"
csv_file_path = os.path.join(output_dir, csv_file_name)

# 3. Pandas DataFrame 생성 (컬럼 하나만 사용)
df_text = pd.DataFrame({
    'transcribed_text': [full_text]
})

# 4. CSV 파일로 저장
df_text.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

# 결과 출력
print("--- 📝 전체 인식 결과 ---")
print(full_text)
print("------------------------")
print(f"\n✅ CSV 저장 완료: '{csv_file_path}'")
print(f"저장된 데이터프레임 미리보기:\n{df_text.head()}")

사용 장치: cuda:0, dtype: torch.float16, 모델: openai/whisper-large-v2


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


'/content/drive/MyDrive/final_project/회의록/1105_오전회의.m4a' 파일 변환 시작...
--- 📝 전체 인식 결과 ---
시작되면 말해주세요 쉽게 구한다는 말은 쉽게 꿀빠하려는 의미보다는 쉽게 더 강력한 데이터를 구해서 우리 모델한테 더 좋은 먹이를 주고 싶다는 연료를 주고 싶다는 마음이 컸어요. 그러다 보니까 실시간 세계 경제 뉴스나 바라다 자동 탱잉을 좀 더 해보자 그리고 팀원들의 어떤 의견 수렴도 전반적으로 그런 기류였던 것 같아서 같이 그냥 동참해서 가는 게 좋을 것 같아서 신속한 의사결정을 위해서 근데 이제 자료를 조사해봤더니 국립국어원에서 상당히 강력하고 풍부한 데이터를 제공해 주는 것 같습니다 근데 이게 지금 음성 형태인지 아니면 그냥 포퍼스라니까 그냥 어떤 그 텍스트 형태인지 좀 더 뜯어 보고 공유 드릴 예정이구요. 그리고 이제 두번째로 제가 지금 염두에 두는게 AI허브의 대화 데이터는 정말 무궁무진하고 많더라구요. 그리고 상황도 다양해요. 예를 들면 긴급 출동 상황, 공항 주변 소음 상황, 공사장 상황, 노이즈가 포함되느냐 안되느냐를 떠나서도 엄청 많더라구요. 아동의 대화 이런거 그래서 우리가 우리에게 필요한 데이터로 보여지는게 많아서 좀 이제 많이 스크랩 해놨으니까 이따 또 한번 리스트업 해서 공유하도록 하고 어쨌든 그래서 결과적으로 저는 바라자 자동 태깅 쪽으로 데이터를 좀 많이 구비해놨다 정도로 볼 수 있고 여기서 이런 데이터는 충분하네요 라고 해서 되면 바로 파이프라인 설계로 넘어가도 저는 괜찮을 것 같습니다 비전적 양식인거 같아서 그 부분은 조금 고민을 안했습니다 안할거 같아서 저도 그래서 두번째로 저도 작가 문체를 조금 찾아봤습니다 작가 문체에 대한 데이터는 어느정도 AI허브에도 있었고 허깅페이스나 깃허브에도 좀 있었던거 같아요 그래서 좀 데이터를 가져왔고 근데 이제 어떻게 좀 진행을 해�가 너무 어려운 것 같아요 저는 문체를 변경한다라는 학습 자체가 너무 어렵고 그걸 어떻게 진행해야 될까는 아무리